In [132]:
from glob import glob
import plantcv as pcv
import cv2
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from tqdm import tqdm

In [134]:
def lab_threshold(img, l_factor = 0.36, b_factor = 0.61, k1 = 7, k2 = 7, closing = True):
    
    lab_img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = lab_img[:,:,0]/255,lab_img[:,:,1]/255, lab_img[:,:,2]/255

    mask = np.zeros(l.shape)
    cond = np.logical_or(l < l_factor, b > b_factor)
    mask[ cond ] = 255

    # remove noise in the background
    kernel = np.ones((k1,k1),np.uint8)
    output = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

    # close retrieved shapes
    if closing:
        kernel = np.ones((k2,k2),np.uint8)
        output = cv2.morphologyEx(output, cv2.MORPH_CLOSE, kernel)
    
    return output

In [143]:
paths = glob(os.getcwd() + '/images/AnalyseLeafArea_SO/*.JPG')
outpath = os.getcwd() + '/images/AnalyseLeafArea_output'
arr = []
for p in tqdm(paths):
    fn = p.split('/')[-1]
    img = cv2.imread(p)
    mask = lab_threshold(img)

    #save output
    cv2.imwrite( outpath + '/mask_' + fn, mask)
    h, w = mask.shape
    tot_pixels = h*w
    
    n_pixels = np.count_nonzero(mask)
    res = n_pixels / tot_pixels
    arr.append([fn, n_pixels, tot_pixels, res])

df = pd.DataFrame(np.array(arr), columns = ['Filename', 'Green pixels', 'Total pixels', 'Coverage'])
df.to_csv('coverage_results.csv')



  1%|          | 21/2736 [00:14<41:35,  1.09it/s]

In [137]:
df

,Filename,Green pixels,Total pixels,Coverage
0,145b_A_SO_IMG_4717_220831_WB_A2.JPG,1523688,4610560,0.3304778595224875
1,145b_A_SO_IMG_4717_220831_WB_A2_Feuilles.JPG,1455962,4610560,0.31578853761799003
2,145b_A_SO_IMG_4717_220831_WB_A2_Tiges.JPG,68236,4610560,0.014799937534702943
3,145b_B_SO_IMG_4718_220831_WB_A2.JPG,1163718,4628480,0.25142552198561946
